In [21]:
# analyze_coverdrive.py
# ======================
# Unified pipeline: single canonical thresholds/verdict + CLI (argparse)
# Produces side-by-side output: left = minimal input, right = black skeleton + labels
# Works from CLI and is safe to run in Jupyter (uses parse_known_args()).
# ======================

# ====== Imports ======
import os
import time
import json
import sys
import argparse
import cv2
import numpy as np
import mediapipe as mp

# ====== Canonical THRESHOLDS (single source of truth) ======
THRESHOLDS = {
    "elbow_angle": {"good": (140, 170), "watch_low": (120, 139), "warn_low": -1, "warn_high": 175},
    "spine_lean":  {"good": (5, 20),    "watch_low": (0, 4),     "warn_low": -999, "warn_high": 30},
    "head_knee_dx":{"good": (-0.06, 0.06), "watch_low": (-0.12, -0.06), "warn_low": -999, "warn_high": 0.12},
    "foot_angle":  {"good": (10, 30),   "watch_low": (0, 9),     "warn_low": -999, "warn_high": 40}
}

def verdict(metric, val):
    """
    Canonical verdict: returns 'good' / 'watch' / 'warn' / 'none'
    Keeps behavior sensible and consistent across code.
    """
    if val is None:
        return "none"
    th = THRESHOLDS.get(metric)
    if not th:
        return "none"
    lo, hi = th["good"]
    if lo <= val <= hi:
        return "good"
    w = th.get("watch_low")
    if w and (w[0] <= val <= w[1]):
        return "watch"
    if val <= th.get("warn_low", -1e9) or val >= th.get("warn_high", 1e9):
        return "warn"
    return "watch"

def percent_to_score(pct):   # pct in [0,1]
    pct = max(0.0, min(1.0, pct))
    return int(round(1 + 9 * pct))

def percent_to_score_10(p):
    """Alternative: map [0..1] to 0..10"""
    return int(round(10 * p))

# small utility to safely fetch landmark by index
def safe_get(lmList, idx):
    try:
        return lmList[idx][1:3]
    except Exception:
        return None

# ==================================
# POSE DETECTOR CLASS (MediaPipe solutions.pose)
# ==================================
class PoseDetector:
    def __init__(self, staticImageMode=False, modelComplexity=1,
                 smoothLandmarks=True, enableSegmentation=False,
                 smoothSegmentation=True, minDetectionConfidence=0.5,
                 minTrackingConfidence=0.5):

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.drawing_styles = mp.solutions.drawing_styles

        self.pose = self.mpPose.Pose(
            static_image_mode=staticImageMode,
            model_complexity=modelComplexity,
            smooth_landmarks=smoothLandmarks,
            enable_segmentation=enableSegmentation,
            smooth_segmentation=smoothSegmentation,
            min_detection_confidence=minDetectionConfidence,
            min_tracking_confidence=minTrackingConfidence
        )

        self.results = None

    def findPose(self, img, draw=True, styled=True):
        if img is None:
            return img
        if img.shape[-1] == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb = np.array(img_rgb, dtype=np.uint8, order="C")
        self.results = self.pose.process(img_rgb)
        if self.results and self.results.pose_landmarks and draw:
            if styled:
                self.mpDraw.draw_landmarks(
                    img,
                    self.results.pose_landmarks,
                    self.mpPose.POSE_CONNECTIONS,
                    landmark_drawing_spec=self.drawing_styles.get_default_pose_landmarks_style()
                )
            else:
                self.mpDraw.draw_landmarks(
                    img,
                    self.results.pose_landmarks,
                    self.mpPose.POSE_CONNECTIONS
                )
        return img

    def findPosition(self, img, draw=False):
        if not self.results or not self.results.pose_landmarks:
            return []
        lmList = []
        h, w, _ = img.shape
        for id, lm in enumerate(self.results.pose_landmarks.landmark):
            cx, cy = int(lm.x * w), int(lm.y * h)
            lmList.append([id, cx, cy])
            if draw:
                cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return lmList

    def getWorldLandmarks(self, frame_idx):
        if not self.results or not self.results.pose_world_landmarks:
            return {"frame": frame_idx, "landmarks": []}
        frame_landmarks = []
        for idx, lm in enumerate(self.results.pose_world_landmarks.landmark):
            frame_landmarks.append({
                "id": idx,
                "x": lm.x,
                "y": lm.y,
                "z": lm.z,
                "visibility": lm.visibility
            })
        return {"frame": frame_idx, "landmarks": frame_landmarks}

# ==================================
# HELPER FUNCTIONS FOR CRICKET METRICS
# ==================================
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    denom = (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    cosine = np.dot(ba, bc) / denom
    angle = np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))
    return float(angle)

def compute_metrics(lmList, image_width):
    metrics = {"elbow_angle": None, "spine_lean": None, "head_knee_dx": None, "foot_angle": None,
               "raw_head_x": None, "raw_knee_x": None}
    if len(lmList) > 0:
        try:
            shoulder = lmList[11][1:3]
            elbow = lmList[13][1:3]
            wrist = lmList[15][1:3]
            metrics["elbow_angle"] = calculate_angle(shoulder, elbow, wrist)

            hip = np.array(lmList[23][1:3])
            shoulder_pt = np.array(lmList[11][1:3])
            vertical_ref = np.array([shoulder_pt[0], hip[1]])
            metrics["spine_lean"] = calculate_angle(hip, shoulder_pt, vertical_ref)

            head_x = lmList[0][1]
            knee_x = lmList[25][1]
            metrics["raw_head_x"] = head_x
            metrics["raw_knee_x"] = knee_x
            dx = (head_x - knee_x) / (image_width + 1e-6)
            metrics["head_knee_dx"] = float(dx)

            ankle = lmList[27][1:3]
            heel = lmList[29][1:3]
            toe = lmList[31][1:3]
            metrics["foot_angle"] = calculate_angle(heel, ankle, toe)
        except Exception:
            pass
    return metrics

def draw_verdicts_on_frame(frame, verdicts, x0=30, y0=50):
    color_map = {"good": (0,200,0), "watch": (0,200,200), "warn": (0,0,255), "no_data": (100,100,100), "none": (100,100,100)}
    dy = 34
    i = 0
    for k, v in verdicts.items():
        color = color_map.get(v, (255,255,255))
        text = f"{k}: {v}"
        cv2.putText(frame, text, (x0, y0 + i*dy), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 6, cv2.LINE_AA)
        cv2.putText(frame, text, (x0, y0 + i*dy), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
        i += 1
    return frame

# ==================================
# EVALUATION, FEEDBACK (unchanged)
# ==================================
def evaluate_shot(all_metrics):
    elbow_series = [m.get("elbow_angle") for m in all_metrics]
    valid_idxs = [i for i,v in enumerate(elbow_series) if v is not None]
    if not valid_idxs:
        impact_idx = None
    else:
        min_pairs = [(i, elbow_series[i]) for i in valid_idxs]
        impact_idx = min(min_pairs, key=lambda x: x[1])[0]

    if impact_idx is None:
        window = all_metrics[-11:] if len(all_metrics)>0 else []
    else:
        start = max(0, impact_idx - 5)
        end = min(len(all_metrics), impact_idx + 6)
        window = all_metrics[start:end]

    metrics_result = {}
    for metric in ["elbow_angle", "spine_lean", "head_knee_dx", "foot_angle"]:
        vals = [f.get(metric) for f in window if f.get(metric) is not None]
        if len(vals) == 0:
            med = None
            pct_good = 0.0
        else:
            med = float(np.median(vals))
            goods = sum(1 for v in vals if verdict(metric, v) == "good")
            pct_good = goods / len(vals)
        metrics_result[metric] = {"median": med, "pct_good": round(pct_good, 3)}

    head_pct = 0.6 * metrics_result["head_knee_dx"]["pct_good"] + 0.4 * metrics_result["spine_lean"]["pct_good"]
    foot_pct = metrics_result["foot_angle"]["pct_good"]
    swing_pct = metrics_result["elbow_angle"]["pct_good"]
    balance_pct = 0.5 * metrics_result["spine_lean"]["pct_good"] + 0.5 * metrics_result["head_knee_dx"]["pct_good"]
    follow_pct = 0.0

    scores = {
        "Head Position": percent_to_score_10(head_pct),
        "Footwork": percent_to_score_10(foot_pct),
        "Swing Control": percent_to_score_10(swing_pct),
        "Balance": percent_to_score_10(balance_pct),
        "Follow-through": percent_to_score_10(follow_pct)
    }

    feedback = {}
    if scores["Head Position"] >= 7:
        feedback["Head Position"] = "Good: head slightly ahead of front knee."
    else:
        feedback["Head Position"] = "Work: push weight forward; try getting head ahead of front knee at impact."

    fa = metrics_result["foot_angle"]["median"]
    if fa is None:
        feedback["Footwork"] = "No foot data. Check visibility of feet in camera."
    elif 10 <= fa <= 30:
        feedback["Footwork"] = "Good: front toe open in ideal range."
    elif fa < 10:
        feedback["Footwork"] = "Watch: front foot too closed; open toe toward mid-off (10°-30°)."
    else:
        feedback["Footwork"] = "Watch: front foot too open; may reduce stability."

    ea = metrics_result["elbow_angle"]["median"]
    if ea is None:
        feedback["Swing Control"] = "No elbow data."
    elif 140 <= ea <= 170:
        feedback["Swing Control"] = "Good: elbow extension in range for a controlled drive."
    elif ea < 140:
        feedback["Swing Control"] = "Work: elbow too cramped — try lengthening front arm."
    else:
        feedback["Swing Control"] = "Watch: elbow nearly locked — risk of reduced shock absorption."

    sl = metrics_result["spine_lean"]["median"]
    if sl is None:
        feedback["Balance"] = "No spine data."
    elif 5 <= sl <= 20:
        feedback["Balance"] = "Good: forward lean supports forward weight transfer."
    elif sl < 5:
        feedback["Balance"] = "Watch: neutral/back lean — get chest slightly over front foot."
    else:
        feedback["Balance"] = "Watch: excessive forward lean — may over-commit."

    evaluation = {
        "impact_index": impact_idx,
        "metrics": metrics_result,
        "scores": scores,
        "feedback": feedback
    }
    return evaluation

def print_evaluation(eval_dict):
    print("\n=== Shot Evaluation ===")
    print(f"Impact frame index: {eval_dict['impact_index']}")
    print("\n-- Metrics (median, %good) --")
    for m, vals in eval_dict["metrics"].items():
        med = vals["median"]
        pct = vals["pct_good"]
        med_str = f"{med:.1f}" if med is not None else "--"
        print(f"{m:12s}: median={med_str}  pct_good={pct*100:.1f}%")
    print("\n-- Scores (0–10) --")
    for cat, score in eval_dict["scores"].items():
        print(f"{cat:15s}: {score}/10")
    print("\n-- Feedback --")
    for cat, fb in eval_dict["feedback"].items():
        print(f"{cat:15s}: {fb}")
    print("========================\n")

# -----------------------------------------
# Phase2 subset skeleton helpers (14 joints)
# -----------------------------------------
mp_pose = mp.solutions.pose
REQ = {
    "head": mp_pose.PoseLandmark.NOSE,
    "ls": mp_pose.PoseLandmark.LEFT_SHOULDER, "rs": mp_pose.PoseLandmark.RIGHT_SHOULDER,
    "le": mp_pose.PoseLandmark.LEFT_ELBOW,    "re": mp_pose.PoseLandmark.RIGHT_ELBOW,
    "lw": mp_pose.PoseLandmark.LEFT_WRIST,    "rw": mp_pose.PoseLandmark.RIGHT_WRIST,
    "lh": mp_pose.PoseLandmark.LEFT_HIP,      "rh": mp_pose.PoseLandmark.RIGHT_HIP,
    "lk": mp_pose.PoseLandmark.LEFT_KNEE,     "rk": mp_pose.PoseLandmark.RIGHT_KNEE,
    "la": mp_pose.PoseLandmark.LEFT_ANKLE,    "ra": mp_pose.PoseLandmark.RIGHT_ANKLE,
}

CONNS = [
    ("ls", "rs"),
    ("ls", "le"), ("le", "lw"),
    ("rs", "re"), ("re", "rw"),
    ("ls", "lh"), ("rs", "rh"),
    ("lh", "lk"), ("lk", "la"),
    ("rh", "rk"), ("rk", "ra"),
]

def draw_subset_skeleton(img, kpts_xy, color=(0,255,0)):
    for name, pt in kpts_xy.items():
        if pt is None: continue
        x, y = pt
        cv2.circle(img, (x, y), 4, color, -1)
    for a, b in CONNS:
        pa, pb = kpts_xy.get(a), kpts_xy.get(b)
        if pa is None or pb is None: continue
        cv2.line(img, pa, pb, color, 2)

def get_req_keypoints(results, w, h, vis_thresh=0.5):
    kpts = {name: None for name in REQ.keys()}
    if not results or not results.pose_landmarks:
        return kpts
    lms = results.pose_landmarks.landmark
    for name, idx in REQ.items():
        lm = lms[idx]
        if lm.visibility is not None and lm.visibility < vis_thresh:
            kpts[name] = None
        else:
            if lm.x < 0 or lm.x > 1 or lm.y < 0 or lm.y > 1:
                kpts[name] = None
            else:
                x = int(np.clip(lm.x, 0, 1) * w)
                y = int(np.clip(lm.y, 0, 1) * h)
                kpts[name] = (x, y)
    return kpts


In [27]:
# ==================================
# MAIN PIPELINE (merged + CLI-friendly)
# ==================================
def analyze_video(input_path="input_videos/input.mp4",
                  output_dir="outputs",
                  base_name="phase2_with_skeleton",
                  codec="mp4v",
                  show_window=False):
    os.makedirs(os.path.dirname(input_path) or ".", exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # auto-increment output filename
    i = 1
    while True:
        output_path = os.path.join(output_dir, f"{base_name}_{i}.mp4")
        if not os.path.exists(output_path):
            break
        i += 1

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"❌ Could not open {input_path}. Please check the path.")

    src_fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) or 640
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 480

    combined_width = width * 2
    combined_height = height

    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_path, fourcc, src_fps, (combined_width, combined_height))

    # --- Minimal logging: metrics CSV (safe, write mode)
    metrics_csv_path = os.path.join(output_dir, "metrics.csv")
    import csv
    csv_file = open(metrics_csv_path, "w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_header = [
        "frame_idx", "time_s",
        "elbow_angle", "spine_lean", "head_knee_dx", "foot_angle",
        "left_wrist_speed_px_per_s", "right_wrist_speed_px_per_s",
        "conf_elbow", "conf_spine", "conf_head_dx", "conf_foot"
    ]
    csv_writer.writerow(csv_header)
    # prev wrist positions (pixel coords) for speed calc
    prev_lw = None
    prev_rw = None

    detector = PoseDetector()
    all_metrics = []
    all_world_landmarks = []

    pTime = time.time()
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        frame_for_detector = frame.copy()
        frame_for_detector = detector.findPose(frame_for_detector, draw=False, styled=False)
        lmList = detector.findPosition(frame_for_detector, draw=False)

        metrics = compute_metrics(lmList, width)

        per_frame_verdicts = {
            "elbow_angle": verdict("elbow_angle", metrics.get("elbow_angle")),
            "spine_lean": verdict("spine_lean", metrics.get("spine_lean")),
            "head_knee_dx": verdict("head_knee_dx", metrics.get("head_knee_dx")),
            "foot_angle": verdict("foot_angle", metrics.get("foot_angle")),
        }

        metrics_record = {"frame_idx": frame_idx, **metrics, "verdicts": per_frame_verdicts}
        all_metrics.append(metrics_record)

        # --- compute simple wrist speeds and per-metric confidences (safe guards)
        time_s = frame_idx / (src_fps + 1e-8)

        def _vis(idx):
            try:
                if detector.results and detector.results.pose_landmarks:
                    return float(detector.results.pose_landmarks.landmark[idx].visibility or 0.0)
            except Exception:
                pass
            return 0.0

        lw = safe_get(lmList, 15)  # left wrist
        rw = safe_get(lmList, 16)  # right wrist

        dt = 1.0 / (src_fps + 1e-8)
        lw_speed = None
        rw_speed = None
        if lw is not None and prev_lw is not None:
            dx = lw[0] - prev_lw[0]
            dy = lw[1] - prev_lw[1]
            lw_speed = ((dx*dx + dy*dy) ** 0.5) / dt
        if rw is not None and prev_rw is not None:
            dx = rw[0] - prev_rw[0]
            dy = rw[1] - prev_rw[1]
            rw_speed = ((dx*dx + dy*dy) ** 0.5) / dt

        prev_lw = lw
        prev_rw = rw

        conf_elbow = min(_vis(11), _vis(13), _vis(15))
        conf_spine = min(_vis(23), _vis(11))
        conf_head_dx = min(_vis(0), _vis(25))
        conf_foot = min(_vis(27), _vis(29), _vis(31))

        csv_row = [
            frame_idx,
            round(time_s, 4),
            round(metrics.get("elbow_angle"), 2) if metrics.get("elbow_angle") is not None else "--",
            round(metrics.get("spine_lean"), 2) if metrics.get("spine_lean") is not None else "--",
            round(metrics.get("head_knee_dx"), 4) if metrics.get("head_knee_dx") is not None else "--",
            round(metrics.get("foot_angle"), 2) if metrics.get("foot_angle") is not None else "--",
            round(lw_speed, 2) if lw_speed is not None else "--",
            round(rw_speed, 2) if rw_speed is not None else "--",
            round(conf_elbow, 3) if conf_elbow is not None else "--",
            round(conf_spine, 3) if conf_spine is not None else "--",
            round(conf_head_dx, 3) if conf_head_dx is not None else "--",
            round(conf_foot, 3) if conf_foot is not None else "--",
        ]
        csv_writer.writerow(csv_row)

        world_landmarks = detector.getWorldLandmarks(frame_idx)
        if world_landmarks["landmarks"]:
            all_world_landmarks.append(world_landmarks)

        skeleton_canvas = np.zeros_like(frame)
        kpts = get_req_keypoints(detector.results, width, height, vis_thresh=0.45)
        draw_subset_skeleton(skeleton_canvas, kpts, color=(0,255,0))

        anomaly = (kpts["lw"] is None) or (kpts["rw"] is None)
        if anomaly:
            cv2.putText(skeleton_canvas, "⚠ Anomaly Detected (wrist occluded)", (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

        cTime = time.time()
        proc_fps = 1.0 / max(1e-6, (cTime - pTime))
        pTime = cTime

        # Right-side overlays
        cv2.putText(skeleton_canvas, f"Frame: {frame_idx}", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(skeleton_canvas, f"Src FPS: {int(round(src_fps))}", (30, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200,200,200), 2, cv2.LINE_AA)
        cv2.putText(skeleton_canvas, f"Proc FPS: {int(proc_fps)}", (30, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

        val_elbow = f"{round(metrics.get('elbow_angle'),1)}" if metrics.get('elbow_angle') is not None else "--"
        val_spine = f"{round(metrics.get('spine_lean'),1)}" if metrics.get('spine_lean') is not None else "--"
        val_headdx = f"{round(metrics.get('head_knee_dx'),3)}" if metrics.get('head_knee_dx') is not None else "--"
        val_foot = f"{round(metrics.get('foot_angle'),1)}" if metrics.get('foot_angle') is not None else "--"

        cv2.putText(skeleton_canvas, f"Elbow: {val_elbow}", (30, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (180,255,180), 2, cv2.LINE_AA)
        cv2.putText(skeleton_canvas, f"Spine: {val_spine}", (30, 160),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (180,255,180), 2, cv2.LINE_AA)
        cv2.putText(skeleton_canvas, f"HeadDx: {val_headdx}", (30, 190),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (180,255,180), 2, cv2.LINE_AA)
        cv2.putText(skeleton_canvas, f"FootAng: {val_foot}", (30, 220),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (180,255,180), 2, cv2.LINE_AA)

        draw_verdicts_on_frame(skeleton_canvas, per_frame_verdicts, x0=30, y0=260)

        left_annotated = frame.copy()
        if len(lmList) > 0:
            nose = safe_get(lmList, 0)
            lk = safe_get(lmList, 25)
            if nose is not None:
                cv2.circle(left_annotated, tuple(nose), 5, (0,255,255), -1)
            if lk is not None:
                cv2.circle(left_annotated, tuple(lk), 5, (255,255,0), -1)

        combined = np.hstack((left_annotated, skeleton_canvas))
        out.write(combined)

        if show_window:
            cv2.imshow("Combined (Left: input, Right: skeleton+labels)", combined)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    out.release()
    if show_window:
        cv2.destroyAllWindows()

    evaluation = evaluate_shot(all_metrics)
    eval_path = os.path.join(output_dir, "evaluation.json")
    wl_path = os.path.join(output_dir, "world_landmarks.json")
    with open(eval_path, "w") as f:
        json.dump(evaluation, f, indent=2)
    with open(wl_path, "w") as f:
        json.dump(all_world_landmarks, f, indent=2)

    # --- also save a short human-readable text summary
    txt_path = os.path.join(output_dir, "evaluation.txt")
    try:
        with open(txt_path, "w") as tf:
            tf.write("Shot Evaluation Summary\n")
            tf.write("=======================\n")
            tf.write(f"Impact frame index: {evaluation.get('impact_index')}\n\n")
            tf.write("-- Scores (0-10) --\n")
            for cat, score in evaluation.get("scores", {}).items():
                tf.write(f"{cat:15s}: {score}/10\n")
            tf.write("\n-- Feedback --\n")
            for cat, fb in evaluation.get("feedback", {}).items():
                tf.write(f"{cat:15s}: {fb}\n")
        print(f"[✅] Wrote human-readable evaluation: {txt_path}")
    except Exception as e:
        print(f"[⚠] Could not write evaluation.txt: {e}")

    # --- close CSV file
    try:
        csv_file.close()
        print(f"[✅] Per-frame metrics saved to: {metrics_csv_path}")
    except Exception:
        pass

    print(f"[✅] Processing complete.")
    print(f"  Video saved: {output_path}")
    print(f"  Evaluation JSON: {eval_path}")
    print(f"  World landmarks JSON: {wl_path}")

    try:
        print_evaluation(evaluation)
    except Exception:
        pass

    if os.path.exists(output_path):
        try:
            from IPython.display import Video, display
            display(Video(output_path, embed=True, width=900, height=400))
        except Exception:
            pass

    return {
        "video": output_path,
        "evaluation": evaluation,
        "world_landmarks": all_world_landmarks
    }

# ==================================
# CLI / Jupyter entrypoint
# ==================================
def build_arg_parser():
    parser = argparse.ArgumentParser(description="Analyze cricket cover-drive video and produce skeleton+metrics.")
    parser.add_argument("--input", "-i", type=str, default="input_videos/input.mp4", help="Input video path")
    parser.add_argument("--outdir", "-o", type=str, default="outputs", help="Output directory")
    parser.add_argument("--basename", "-b", type=str, default="phase2_with_skeleton", help="Base name for output file (auto-increment appended)")
    parser.add_argument("--codec", type=str, default="mp4v", help="FourCC codec (e.g. mp4v or avc1)")
    parser.add_argument("--show", action="store_true", help="Show live window during processing")
    return parser

def main():
    parser = build_arg_parser()
    # Use parse_known_args so Jupyter's argv doesn't break the parser
    args, _ = parser.parse_known_args()
    analyze_video(input_path=args.input, output_dir=args.outdir, base_name=args.basename, codec=args.codec, show_window=args.show)

if __name__ == "__main__":
    main()



[✅] Wrote human-readable evaluation: outputs\evaluation.txt
[✅] Per-frame metrics saved to: outputs\metrics.csv
[✅] Processing complete.
  Video saved: outputs\phase2_with_skeleton_1.mp4
  Evaluation JSON: outputs\evaluation.json
  World landmarks JSON: outputs\world_landmarks.json

=== Shot Evaluation ===
Impact frame index: 106

-- Metrics (median, %good) --
elbow_angle : median=90.0  pct_good=0.0%
spine_lean  : median=18.1  pct_good=90.9%
head_knee_dx: median=-0.1  pct_good=0.0%
foot_angle  : median=41.1  pct_good=9.1%

-- Scores (0–10) --
Head Position  : 4/10
Footwork       : 1/10
Swing Control  : 0/10
Balance        : 5/10
Follow-through : 0/10

-- Feedback --
Head Position  : Work: push weight forward; try getting head ahead of front knee at impact.
Footwork       : Watch: front foot too open; may reduce stability.
Swing Control  : Work: elbow too cramped — try lengthening front arm.
Balance        : Good: forward lean supports forward weight transfer.

